In [184]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [186]:
#Configura el Modelo Base excluyendo la última capa de clasificación
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [187]:
#Congelar las Capas del Modelo Base para que no se actualicen durante el entrenamiento
for layer in base_model.layers:
    layer.trainable = False

In [190]:
from tensorflow.keras.layers import Dropout

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Capa Dropout adicional
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)  # Otra capa Dropout
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [192]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [194]:
import zipfile
import os

# Ruta del archivo y el directorio de destino
zip_path = '/home/adrian/Descargas/archive (1).zip'

extract_dir = '/home/adrian/Descargas/cats_vs_noncats'

# Extraer el archivo zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Listar los archivos y directorios para ver cómo están organizados
extracted_files = os.listdir(extract_dir)
extracted_files

['catvnoncat']

In [196]:
catvnoncat_dir = os.path.join(extract_dir, 'catvnoncat')
subdirectories = os.listdir(catvnoncat_dir)
subdirectories

['train_catvnoncat.h5', 'test_catvnoncat.h5']

In [198]:
catvnoncat_dir = os.path.join(extract_dir, 'catvnoncat')

In [200]:
import h5py
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input

def load_dataset(file_path):
    with h5py.File(file_path, 'r') as h5file:
        images = np.array(h5file['train_set_x'] if 'train_set_x' in h5file else h5file['test_set_x'])
        labels = np.array(h5file['train_set_y'] if 'train_set_y' in h5file else h5file['test_set_y'])
        class_list = np.array(h5file['list_classes'])
    return images, labels, class_list

# Rutas completas de los archivos de entrenamiento y prueba
train_file_path = '/home/adrian/Descargas/cats_vs_noncats/catvnoncat/train_catvnoncat.h5'
test_file_path = '/home/adrian/Descargas/cats_vs_noncats/catvnoncat/test_catvnoncat.h5'

# Cargar el conjunto de datos de entrenamiento y prueba
train_images, train_labels, classes = load_dataset(train_file_path)
test_images, test_labels, _ = load_dataset(test_file_path)

# Redimensionar las imágenes con padding y normalización adecuada para ResNet50
train_images_resized = tf.image.resize_with_pad(train_images, 224, 224)
test_images_resized = tf.image.resize_with_pad(test_images, 224, 224)

# Normalizar las imágenes al formato que espera ResNet50
train_images_resized = preprocess_input(train_images_resized)
test_images_resized = preprocess_input(test_images_resized)

print(f"Training images resized shape: {train_images_resized.shape}")
print(f"Test images resized shape: {test_images_resized.shape}")

Training images resized shape: (209, 224, 224, 3)
Test images resized shape: (50, 224, 224, 3)


In [202]:
# Verificar las etiquetas únicas en el conjunto de entrenamiento y prueba
unique_train_labels = np.unique(train_labels)
unique_test_labels = np.unique(test_labels)

print("Etiquetas únicas en el conjunto de entrenamiento:", unique_train_labels)
print("Etiquetas únicas en el conjunto de prueba:", unique_test_labels)

Etiquetas únicas en el conjunto de entrenamiento: [0 1]
Etiquetas únicas en el conjunto de prueba: [0 1]


In [204]:
history = model.fit(
    train_images_resized, train_labels,
    validation_data=(test_images_resized, test_labels),
    epochs=10,
    batch_size=32
)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 807ms/step - accuracy: 0.5935 - loss: 0.7248 - val_accuracy: 0.6600 - val_loss: 0.5213
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 653ms/step - accuracy: 0.9140 - loss: 0.2848 - val_accuracy: 1.0000 - val_loss: 0.1071
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 660ms/step - accuracy: 0.9475 - loss: 0.1706 - val_accuracy: 1.0000 - val_loss: 0.0525
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 658ms/step - accuracy: 0.9348 - loss: 0.1484 - val_accuracy: 0.9800 - val_loss: 0.0534
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 673ms/step - accuracy: 0.9875 - loss: 0.0699 - val_accuracy: 1.0000 - val_loss: 0.0365
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 696ms/step - accuracy: 0.9745 - loss: 0.0685 - val_accuracy: 1.0000 - val_loss: 0.0235
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 702ms/step - accuracy: 0.9862 - loss: 0.0467 - val_accuracy: 1.0000 - val_loss: 0.0222
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 680ms/step - accuracy: 0.9844 - loss: 0.0524 - val_accuracy: 1.0000 - val_loss:

In [206]:
loss, accuracy = model.evaluate(test_images_resized, test_labels)
print(f'Pérdida: {loss}')
print(f'Precisión: {accuracy}')


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 303ms/step - accuracy: 1.0000 - loss: 0.0172
Pérdida: 0.020186210051178932
Precisión: 1.0


In [210]:
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
import tensorflow as tf

def predict_image(img_array):
    # Redimensionar la imagen
    img_resized = tf.image.resize(img_array, (224, 224))

    # Preprocesar la imagen de la misma forma que durante el entrenamiento
    img_resized = preprocess_input(img_resized)
    
    # Añadir dimensión de lote
    img_resized = np.expand_dims(img_resized, axis=0)

    # Hacer la predicción
    prediction = model.predict(img_resized)[0][0]

    # Calcular el porcentaje de confianza
    confidence = prediction * 100

    # Interpretar y mostrar el resultado
    if prediction > 0.5:
        print(f"Es un gato ({confidence:.2f}% de confianza)")
    else:
        print(f"No es un gato ({100 - confidence:.2f}% de confianza)")
predict_image(test_images[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Es un gato (99.66% de confianza)


In [212]:
def load_and_prepare_image(img_path):
    # Cargar la imagen y redimensionarla a 224x224
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)  # Convertir la imagen en un array

    # Aplicar preprocess_input para que el array esté en el rango que espera ResNet50
    img_array = preprocess_input(img_array)
    
    # Añadir una dimensión de lote
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [249]:

def predict_images_in_folder(model, folder_path):
    # Iterar sobre cada archivo en la carpeta
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        
        # Asegurarse de que el archivo es una imagen
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            # Preparar la imagen
            prepared_image = load_and_prepare_image(img_path)

            # Hacer la predicción
            prediction = model.predict(prepared_image)[0][0]
            
            # Calcular el porcentaje de confianza
            confidence = prediction * 100
            print(f"{filename}: Es un gato ({confidence:.2f}% de confianza)")
            
# Ejecutar la función para predecir imágenes en una carpeta
predict_images_in_folder(model, 'img')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
gato_disfraz_rey.jpg: Es un gato (90.75% de confianza)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
istockphoto-941224542-612x612.jpg: Es un gato (5.43% de confianza)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
origami_gato.jpeg: Es un gato (96.44% de confianza)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
mesa.png: Es un gato (1.33% de confianza)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
descarga.jpeg: Es un gato (98.17% de confianza)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
muchos_gato_peque.jpg: Es un gato (96.94% de confianza)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
patito_goma.jpg: Es un gato (5.33% de confianza)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
gato_dibujo.jpg: Es un gato (0.66% de confianza)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
gatito_heterogeneo.jpeg: Es un gato (99.95% de confianza)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
gato_peque2.jpg: Es un gato (98.19% de confianza)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
gato_peque.png: Es un gato (87.59% de confianza)
1/1 ━━━

In [233]:
model.save("modelo_gatos_no_gatos.h5")
model.save("modelo_gatos_no_gatos.keras")
